# 04 - Spatial Regression Models

This notebook fits spatial regression models to examine the relationship between pain/distress metrics and Trump voting, accounting for spatial dependence.

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Spatial regression
import libpysal
from libpysal.weights import Queen
import spreg
from spreg import OLS, ML_Lag, ML_Error, GM_Lag
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Setup
project_root = Path.cwd().parent
data_processed = project_root / 'data' / 'processed'
reports = project_root / 'reports'

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

## 1. Load Data and Prepare Variables

In [ ]:
# Load processed data
gdf = gpd.read_file(data_processed / 'counties_analysis.geojson')

# Drop missing values for key variables
model_vars = [
    'trump_share_2016',
    'freq_phys_distress_pct',
    'od_1316_rate',
    'rucc',
    'ba_plus_pct',
    'median_income',
    'unemp_rate',
    'pct_65plus',
    'pct_nonwhite'
]

# Create analysis dataset
df_model = gdf[model_vars + ['fips', 'NAME', 'geometry']].dropna()
print(f"Analysis sample: {len(df_model)} counties (from {len(gdf)} total)")

# Standardize variables for easier interpretation
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
continuous_vars = ['freq_phys_distress_pct', 'od_1316_rate', 'ba_plus_pct', 
                   'median_income', 'unemp_rate', 'pct_65plus', 'pct_nonwhite']

df_model_std = df_model.copy()
df_model_std[continuous_vars] = scaler.fit_transform(df_model[continuous_vars])

print("\nStandardized variables (mean=0, std=1)")

## 2. Create Spatial Weights

In [ ]:
# Create Queen contiguity weights
w = Queen.from_dataframe(df_model_std, use_index=False)
w.transform = 'r'  # Row-standardize

print(f"Spatial weights summary:")
print(f"  Number of observations: {w.n}")
print(f"  Average neighbors: {w.mean_neighbors:.2f}")
print(f"  Min neighbors: {w.min_neighbors}")
print(f"  Max neighbors: {w.max_neighbors}")

# Check for islands
if len(w.islands) > 0:
    print(f"\nWarning: {len(w.islands)} islands detected")
    # Remove islands for model stability
    df_model_std = df_model_std[~df_model_std.index.isin(w.islands)]
    w = Queen.from_dataframe(df_model_std, use_index=False)
    w.transform = 'r'
    print(f"After removing islands: {len(df_model_std)} counties")

## 3. OLS Baseline Model

In [ ]:
# Prepare variables for spreg
y = df_model_std[['trump_share_2016']].values
y_name = 'trump_share_2016'

# Model 1: Physical distress as main predictor
X1 = df_model_std[[
    'freq_phys_distress_pct',
    'rucc',
    'ba_plus_pct',
    'median_income',
    'unemp_rate',
    'pct_65plus',
    'pct_nonwhite'
]].values

X1_names = [
    'const',
    'phys_distress',
    'rucc',
    'ba_plus',
    'income',
    'unemploy',
    'age_65plus',
    'nonwhite'
]

# Fit OLS model
ols1 = OLS(y, X1, w=w, name_y=y_name, name_x=X1_names, 
           name_w='queen', name_ds='counties')

print(ols1.summary)

## 4. Spatial Diagnostics

In [ ]:
def spatial_diagnostics(model):
    """Extract and display spatial diagnostics from OLS model"""
    
    print("\nSPATIAL DIAGNOSTICS")
    print("="*50)
    
    # Moran's I of residuals
    print(f"Moran's I (residuals): {model.moran_res[0]:.4f}")
    print(f"  z-score: {model.moran_res[1]:.4f}")
    print(f"  p-value: {model.moran_res[2]:.4f}")
    
    print("\nLagrange Multiplier Tests:")
    print(f"LM Lag:")
    print(f"  Statistic: {model.lm_lag[0]:.4f}")
    print(f"  p-value: {model.lm_lag[1]:.4f}")
    
    print(f"\nLM Error:")
    print(f"  Statistic: {model.lm_error[0]:.4f}")
    print(f"  p-value: {model.lm_error[1]:.4f}")
    
    print(f"\nRobust LM Lag:")
    print(f"  Statistic: {model.rlm_lag[0]:.4f}")
    print(f"  p-value: {model.rlm_lag[1]:.4f}")
    
    print(f"\nRobust LM Error:")
    print(f"  Statistic: {model.rlm_error[0]:.4f}")
    print(f"  p-value: {model.rlm_error[1]:.4f}")
    
    # Decision rule
    print("\nMODEL SELECTION GUIDANCE:")
    if model.lm_lag[1] < 0.05 and model.lm_error[1] >= 0.05:
        print("→ Use Spatial Lag Model")
    elif model.lm_error[1] < 0.05 and model.lm_lag[1] >= 0.05:
        print("→ Use Spatial Error Model")
    elif model.lm_lag[1] < 0.05 and model.lm_error[1] < 0.05:
        # Both significant, use robust tests
        if model.rlm_lag[1] < model.rlm_error[1]:
            print("→ Use Spatial Lag Model (based on robust tests)")
        else:
            print("→ Use Spatial Error Model (based on robust tests)")
    else:
        print("→ No significant spatial dependence, OLS may be adequate")

spatial_diagnostics(ols1)

## 5. Spatial Lag Model

In [ ]:
# Fit Spatial Lag Model
lag1 = ML_Lag(y, X1, w=w, name_y=y_name, name_x=X1_names,
              name_w='queen', name_ds='counties')

print(lag1.summary)

# Extract key results
print("\n" + "="*50)
print("KEY RESULTS - SPATIAL LAG MODEL")
print("="*50)
print(f"Spatial lag coefficient (ρ): {lag1.rho:.4f}")
print(f"Physical distress coefficient: {lag1.betas[1][0]:.4f}")
print(f"  Standard error: {lag1.std_err[1]:.4f}")
print(f"  z-value: {lag1.z_stat[1][0]:.4f}")
print(f"  p-value: {lag1.z_stat[1][1]:.4f}")
print(f"\nPseudo R²: {lag1.pr2:.4f}")
print(f"Log-likelihood: {lag1.logll:.2f}")
print(f"AIC: {lag1.aic:.2f}")

## 6. Spatial Error Model

In [ ]:
# Fit Spatial Error Model
error1 = ML_Error(y, X1, w=w, name_y=y_name, name_x=X1_names,
                  name_w='queen', name_ds='counties')

print(error1.summary)

# Extract key results
print("\n" + "="*50)
print("KEY RESULTS - SPATIAL ERROR MODEL")
print("="*50)
print(f"Spatial error coefficient (λ): {error1.lam:.4f}")
print(f"Physical distress coefficient: {error1.betas[1][0]:.4f}")
print(f"  Standard error: {error1.std_err[1]:.4f}")
print(f"  z-value: {error1.z_stat[1][0]:.4f}")
print(f"  p-value: {error1.z_stat[1][1]:.4f}")
print(f"\nPseudo R²: {error1.pr2:.4f}")
print(f"Log-likelihood: {error1.logll:.2f}")
print(f"AIC: {error1.aic:.2f}")

## 7. Model Comparison

In [ ]:
def compare_models(ols_model, lag_model, error_model):
    """Compare OLS, spatial lag, and spatial error models"""
    
    # Create comparison dataframe
    comparison = pd.DataFrame({
        'OLS': [
            ols_model.r2,
            ols_model.logll,
            ols_model.aic,
            ols_model.schwarz,
            ols_model.betas[1][0],
            ols_model.std_err[1],
            np.nan,
            np.nan
        ],
        'Spatial Lag': [
            lag_model.pr2,
            lag_model.logll,
            lag_model.aic,
            lag_model.schwarz,
            lag_model.betas[1][0],
            lag_model.std_err[1],
            lag_model.rho,
            np.nan
        ],
        'Spatial Error': [
            error_model.pr2,
            error_model.logll,
            error_model.aic,
            error_model.schwarz,
            error_model.betas[1][0],
            error_model.std_err[1],
            np.nan,
            error_model.lam
        ]
    }, index=['R²/Pseudo-R²', 'Log-likelihood', 'AIC', 'BIC', 
              'Distress Coef', 'Distress SE', 'ρ (lag)', 'λ (error)'])
    
    print("\nMODEL COMPARISON")
    print("="*60)
    print(comparison.round(4))
    
    # Likelihood ratio tests
    lr_lag = 2 * (lag_model.logll - ols_model.logll)
    lr_error = 2 * (error_model.logll - ols_model.logll)
    
    from scipy import stats
    p_lag = stats.chi2.sf(lr_lag, 1)
    p_error = stats.chi2.sf(lr_error, 1)
    
    print("\n" + "="*60)
    print("LIKELIHOOD RATIO TESTS")
    print("="*60)
    print(f"Spatial Lag vs OLS:")
    print(f"  LR statistic: {lr_lag:.4f}")
    print(f"  p-value: {p_lag:.4f}")
    print(f"\nSpatial Error vs OLS:")
    print(f"  LR statistic: {lr_error:.4f}")
    print(f"  p-value: {p_error:.4f}")
    
    # Best model based on AIC
    best_model = comparison.loc['AIC'].idxmin()
    print(f"\nBest model by AIC: {best_model}")
    
    return comparison

model_comparison = compare_models(ols1, lag1, error1)

## 8. Robustness Checks

In [ ]:
# Model 2: Using overdose rate instead of physical distress
X2 = df_model_std[[
    'od_1316_rate',  # Different pain proxy
    'rucc',
    'ba_plus_pct',
    'median_income',
    'unemp_rate',
    'pct_65plus',
    'pct_nonwhite'
]].values

X2_names = ['const', 'overdose_rate'] + X1_names[2:]

# Fit models with overdose rate
ols2 = OLS(y, X2, w=w, name_y=y_name, name_x=X2_names)
lag2 = ML_Lag(y, X2, w=w, name_y=y_name, name_x=X2_names)

print("\nROBUSTNESS CHECK: OVERDOSE RATE AS PAIN PROXY")
print("="*50)
print(f"OLS coefficient: {ols2.betas[1][0]:.4f} (p={ols2.t_stat[1][1]:.4f})")
print(f"Spatial Lag coefficient: {lag2.betas[1][0]:.4f} (p={lag2.z_stat[1][1]:.4f})")
print(f"Spatial lag ρ: {lag2.rho:.4f}")

In [ ]:
# Model 3: Interaction with rurality
df_model_std['distress_x_rural'] = df_model_std['freq_phys_distress_pct'] * df_model_std['rucc']

X3 = df_model_std[[
    'freq_phys_distress_pct',
    'rucc',
    'distress_x_rural',  # Interaction term
    'ba_plus_pct',
    'median_income',
    'unemp_rate',
    'pct_65plus',
    'pct_nonwhite'
]].values

X3_names = ['const', 'phys_distress', 'rucc', 'distress_x_rural'] + X1_names[3:]

# Fit interaction model
lag3 = ML_Lag(y, X3, w=w, name_y=y_name, name_x=X3_names)

print("\nROBUSTNESS CHECK: INTERACTION WITH RURALITY")
print("="*50)
print(f"Physical distress (main): {lag3.betas[1][0]:.4f} (p={lag3.z_stat[1][1]:.4f})")
print(f"RUCC (main): {lag3.betas[2][0]:.4f} (p={lag3.z_stat[2][1]:.4f})")
print(f"Interaction: {lag3.betas[3][0]:.4f} (p={lag3.z_stat[3][1]:.4f})")
print(f"Spatial lag ρ: {lag3.rho:.4f}")

## 9. Visualize Model Results

In [ ]:
def plot_coefficients(models_dict):
    """Plot coefficient comparison across models"""
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # Extract coefficients and standard errors
    coef_data = []
    for name, model in models_dict.items():
        for i, var_name in enumerate(model['var_names'][1:], 1):  # Skip constant
            coef_data.append({
                'Model': name,
                'Variable': var_name,
                'Coefficient': model['model'].betas[i][0],
                'SE': model['model'].std_err[i],
                'Lower': model['model'].betas[i][0] - 1.96 * model['model'].std_err[i],
                'Upper': model['model'].betas[i][0] + 1.96 * model['model'].std_err[i]
            })
    
    coef_df = pd.DataFrame(coef_data)
    
    # Plot 1: Physical distress coefficient across models
    distress_coef = coef_df[coef_df['Variable'].str.contains('distress')]
    
    ax1 = axes[0]
    x = range(len(distress_coef))
    ax1.errorbar(x, distress_coef['Coefficient'], 
                 yerr=1.96*distress_coef['SE'], 
                 fmt='o', capsize=5, capthick=2, markersize=8)
    ax1.set_xticks(x)
    ax1.set_xticklabels(distress_coef['Model'], rotation=45)
    ax1.axhline(y=0, color='red', linestyle='--', alpha=0.5)
    ax1.set_ylabel('Coefficient (95% CI)')
    ax1.set_title('Physical Distress Effect Across Models')
    ax1.grid(True, alpha=0.3)
    
    # Plot 2: All coefficients for best model
    best_model_coef = coef_df[coef_df['Model'] == 'Spatial Lag']
    
    ax2 = axes[1]
    y_pos = range(len(best_model_coef))
    ax2.barh(y_pos, best_model_coef['Coefficient'], xerr=1.96*best_model_coef['SE'],
             capsize=3, color='steelblue', alpha=0.7)
    ax2.set_yticks(y_pos)
    ax2.set_yticklabels(best_model_coef['Variable'])
    ax2.axvline(x=0, color='red', linestyle='--', alpha=0.5)
    ax2.set_xlabel('Standardized Coefficient (95% CI)')
    ax2.set_title('Spatial Lag Model - All Coefficients')
    ax2.grid(True, alpha=0.3, axis='x')
    
    plt.tight_layout()
    plt.savefig(reports / 'figures/model_coefficients.png', dpi=150, bbox_inches='tight')
    plt.show()

# Create models dictionary
models_for_plot = {
    'OLS': {'model': ols1, 'var_names': X1_names},
    'Spatial Lag': {'model': lag1, 'var_names': X1_names},
    'Spatial Error': {'model': error1, 'var_names': X1_names}
}

plot_coefficients(models_for_plot)

In [ ]:
def plot_residual_maps(gdf, ols_model, spatial_model):
    """Plot residual maps to check for remaining spatial patterns"""
    
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Add residuals to geodataframe
    gdf_plot = gdf.copy()
    gdf_plot = gdf_plot.iloc[:len(ols_model.u)]  # Match model sample
    gdf_plot['ols_resid'] = ols_model.u
    gdf_plot['spatial_resid'] = spatial_model.u
    
    # OLS residuals
    gdf_plot.plot(column='ols_resid', scheme='quantiles', k=5, 
                  cmap='RdBu_r', edgecolor='white', linewidth=0.1, 
                  ax=axes[0], legend=True)
    axes[0].set_title('OLS Model Residuals')
    axes[0].axis('off')
    
    # Spatial model residuals
    gdf_plot.plot(column='spatial_resid', scheme='quantiles', k=5,
                  cmap='RdBu_r', edgecolor='white', linewidth=0.1,
                  ax=axes[1], legend=True)
    axes[1].set_title('Spatial Lag Model Residuals')
    axes[1].axis('off')
    
    plt.tight_layout()
    plt.savefig(reports / 'figures/residual_maps.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    # Calculate Moran's I of residuals
    from esda.moran import Moran
    mi_ols = Moran(gdf_plot['ols_resid'].values, w)
    mi_spatial = Moran(gdf_plot['spatial_resid'].values, w)
    
    print("\nRESIDUAL SPATIAL AUTOCORRELATION")
    print("="*50)
    print(f"OLS residuals Moran's I: {mi_ols.I:.4f} (p={mi_ols.p_norm:.4f})")
    print(f"Spatial Lag residuals Moran's I: {mi_spatial.I:.4f} (p={mi_spatial.p_norm:.4f})")

# plot_residual_maps(df_model_std, ols1, lag1)

## 10. Export Model Results

In [ ]:
def export_model_results(model, model_name, gdf):
    """Export model results for web visualization"""
    
    # Add fitted values and residuals to geodataframe
    gdf_export = gdf.iloc[:len(model.u)].copy()
    gdf_export[f'{model_name}_fitted'] = model.predy
    gdf_export[f'{model_name}_residual'] = model.u
    
    # Create results summary
    results = {
        'model': model_name,
        'n_obs': model.n,
        'coefficients': {},
        'diagnostics': {}
    }
    
    # Add coefficients
    for i, var_name in enumerate(X1_names):
        results['coefficients'][var_name] = {
            'estimate': float(model.betas[i][0]),
            'std_error': float(model.std_err[i]),
            'z_value': float(model.z_stat[i][0]) if hasattr(model, 'z_stat') else float(model.t_stat[i][0]),
            'p_value': float(model.z_stat[i][1]) if hasattr(model, 'z_stat') else float(model.t_stat[i][1])
        }
    
    # Add diagnostics
    if hasattr(model, 'rho'):
        results['diagnostics']['spatial_lag_rho'] = float(model.rho)
    if hasattr(model, 'lam'):
        results['diagnostics']['spatial_error_lambda'] = float(model.lam)
    
    results['diagnostics']['log_likelihood'] = float(model.logll)
    results['diagnostics']['aic'] = float(model.aic)
    results['diagnostics']['r_squared'] = float(model.r2) if hasattr(model, 'r2') else float(model.pr2)
    
    # Save results
    import json
    with open(project_root / 'web' / 'assets' / f'{model_name}_results.json', 'w') as f:
        json.dump(results, f, indent=2)
    
    print(f"Model results exported for {model_name}")
    return gdf_export

# Export results
# gdf_with_results = export_model_results(lag1, 'spatial_lag', df_model_std)
# gdf_with_results.to_file(project_root / 'web' / 'assets' / 'counties_with_model.geojson', driver='GeoJSON')